# Interactive mapping of the project site

Following field mission from the 29/06/2022 to the 02/07/2022, this project aims at proposing an interactive cartography of the project site, project activities and their impact.

## Mapping of the project site
The map below presents the project area by mapping geolocaized photos that have been taken on site, in order to help the reader to get familiar with the site environment

By left-clicking on any point, it will display a photography of the site at this point. The photographies can be downloaded by right clicking and saving.

This map will later be completed with more photos of the site, and the  photos will eventually be classified (infrastructures such as floodgate, cultural heritage, landscape, etc.)

The extended project area will be added to the map below.

In [4]:
import xyzservices.providers as xyz
import geopandas as gpd
import fiona
import json

from os.path import exists
from pathlib import Path

cpnt_geopkg_list=[]
cpnt_files_names_list=[]
cpnt_geojson_list=[]
data_cpnt_geojson_list=[]

photos_files_names=["AbandonedPonds","Gates","MangroveArea","PondsClosetoMangrove","RestoredMangrove","SanjiangPark","Villages"]
photos_geopkg_list=[]
photos_geojson_list=[]
data_photos_geojson_list=[]

#open gpkg containing photos georeferences
for gpkgName in photos_files_names:
    for layername in fiona.listlayers(r'Vectors/Photos sites/'+gpkgName+'.gpkg'):
        photos_geopkg_list.append(gpd.read_file(r'Vectors/Photos sites/'+gpkgName+'.gpkg', layer=layername))
        
        
#convert to geojson (needed for on_click  popup method)
i=0
for i in range(len(photos_geopkg_list)):
    if exists('Vectors/Photos sites/geojson/'+photos_files_names[i]+'.geojson')==False:
        photos_geojson_list.append(photos_geopkg_list[i].to_file('Vectors/Photos sites/geojson/'+photos_files_names[i]+'.geojson', driver='GeoJSON')) 

    with open('Vectors/Photos sites/geojson/'+photos_files_names[i]+'.geojson', 'r') as f:
        data_photos_geojson_list.append(json.load(f))

#print(len(data_photos_geojson_list)) - test

#open gpkg containing components by scanning directory
#assign directory
directory = 'Vectors/Components'

# iterate over files in that directory
files = Path(directory).glob('*')
for file in files:
    #print(file)
    if file.suffix == '.gpkg':
        for layername in fiona.listlayers(file):
            if gpd.read_file(file, layer=layername).empty==False:
                cpnt_files_names_list.append(layername)
                #Create a table containing the GeoDataFrame files in the good CRS
                cpnt_geopkg_list.append(gpd.read_file(file, layer=layername).to_crs(4326))
            
for i in range(len(cpnt_files_names_list)):
    if exists('Vectors/Components/geojson/'+cpnt_files_names_list[i]+'.geojson')==False:
        cpnt_geojson_list.append(cpnt_geopkg_list[i].to_file('Vectors/Components/geojson/'+cpnt_files_names_list[i]+'.geojson', driver='GeoJSON')) 

    with open('Vectors/Components/geojson/'+cpnt_files_names_list[i]+'.geojson', 'r') as f:
        data_cpnt_geojson_list.append(json.load(f))


In [2]:
from ipywidgets import Label, Layout, Output, Image, Button, Box, HTML
from IPython.display import display
from ipyleaflet import Map, GeoData, basemaps, basemap_to_tiles, LayersControl, Marker, Popup, GeoJSON
import random
from PIL import Image as ImagePIL

styled_photos_geojson_list=[]

#import the GoogleSatellite xyz tileset
#tiles=xyz.GoogleSatellite
tiles={'url': 'http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}', 'max_zoom': 18, 'attribution': '(C) Google Maps', 'name': 'GoogleSatellite'}

#Coordinates of center of the bay (WGS)
center_bay=(19.96323,110.61723)

#geo_data = GeoData(geo_data = geopkg)


#create map
m = Map(
    basemap=basemap_to_tiles(tiles),
    center=center_bay,
    zoom=12,
    scroll_wheel_zoom=True
    )

#create styled geojson
for i in range(len(data_photos_geojson_list)):
    styled_photos_geojson_list.append(GeoJSON(
        data=data_photos_geojson_list[i],
        point_style={
            'color': 'orange','opacity': 1, 'dashArray': '0', 'fillOpacity': 0.1, 'weight': 1,'radius':5
        },
        hover_style={
            'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
        }
    ))

#print(len(styled_photos_geojson_list)) - test    
    
#define the handle_click (what happens when on_click is called - i.e. callback func)
def handle_click(feature, **kwargs):

#open image with PIL to get the image size (to define the box)    
    im = ImagePIL.open("IMG/"+feature['properties']['filename']+".jpg")
    ratio = im.size[0]/im.size[1]
    des_width_pic = 500
#call    
    box_layout = Layout(display='flex',
#                        flex_flow='column',
#                        align_items='stretch',
#                        border='solid',
                        width= str(des_width_pic)+'px',
                        height=str(des_width_pic/ratio)+'px')
    
    
    with open("IMG/"+feature['properties']['filename']+".jpg", "rb") as f:
        image = f.read()
    items = Image(
        value=image,
        format ='jpg'
        )
    items.layout.object_fit="cover"
    box = Box(children=[items], layout=box_layout)
    
    #define the popup that will appear
    popup_click = Popup(
        location=(feature['geometry']['coordinates'][1],feature['geometry']['coordinates'][0]),
        max_width=1000,
        child=box,
        close_button=True,

    )

    m.add_layer(popup_click)

for i in range(len(styled_photos_geojson_list)):
    #attach on click action to geojson
    styled_photos_geojson_list[i].on_click(handle_click)

    #add geojson
    m.add_layer(styled_photos_geojson_list[i])

#add layers control
m.add_control(LayersControl())

m




Map(center=[19.96323, 110.61723], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

## Mapping of the project activities

The map below presents the main sub-components of project. The name of the sub-component will appear when hovering over the component on the map.

For component where the extent of the activities can be estimated the map will be update to reflect the explicit

In addition to the component name, a description of the activities will be added to allow smooth navigation of the project content.

Component 1 is represented in Yellow and Component 2 in red. Component 3 is mainly concerned with soft activities dedicated to the support of Components 1 and 2.

In [5]:
styled_cpnt_geojson_list=[]

mb = Map(
    basemap=basemap_to_tiles(tiles),
    center=center_bay,
    zoom=12,
    scroll_wheel_zoom=True
    )

def feature_styling(feature):
    feature["SUBCPNT_NAME"]

for i in range(len(data_cpnt_geojson_list)):
    styled_cpnt_geojson_list.append(GeoJSON(
        data=data_cpnt_geojson_list[i],
        point_style={
          'color': 'red', 'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.5, 'weight': 1,'radius':5, 'use': 'rectangle'
        },
        
        hover_style={
            'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
        }
    ))
    

#change the style for component 2
def component_color(feature):
    if feature["properties"]["CPNT ID"] == 1:
        return {
            'color': 'yellow', "fillOpacity":0.5
        }
    elif feature["geometry"]["type"] == "MultiLineString":
        return {
            'color': 'green', "fillOpacity":1
        }

styled_cpnt_geojson_list[1].style_callback=component_color

styled_cpnt_geojson_list[0].style={'color': 'yellow', "fillOpacity":0.5}
#

def hover_handler(feature,**kwargs):
#    if feature["properties"]["CPNT ID"] == 1:
#        styled_cpnt_geojson_list[0]._handle_mouse_events.dispatch
#        mb.add_layer(styled_photos_geojson_list[0])
    message1 = HTML()
    message1.value = str(feature["properties"]["CPNT ID"])+"."+str(feature["properties"]["SUBCPNT_ID"])+" "+feature["properties"]["SUBCPNT_NAME"]   
    popup_hover = Popup(
    location=(feature['geometry']['coordinates'][1],feature['geometry']['coordinates'][0]),
    max_width=1000,
    child=message1,
    close_button=True,
    auto_close=True
    )

    mb.add_layer(popup_hover)

        
#    print(styled_cpnt_geojson_list[2]._hover_callbacks.register_callback(mb.add_layer(styled_photos_geojson_list[1])))
#    styled_cpnt_geojson_list[2].on_mouseout(mb.remove_layer(styled_photos_geojson_list[1]))

   

styled_cpnt_geojson_list[1].on_hover(hover_handler)


#if styled_photos_geojson_list[1] in mb.layers:      
#        mb.remove_layer(styled_photos_geojson_list[1])
#4

#def mouse_out(feature,**kwargs):
#    if feature["properties"]["CPNT ID"] == 1:
#        print(styled_cpnt_geojson_list[2]._hover_callbacks.register_callback)
#        mb.add_layer(styled_photos_geojson_list[1])
        
#styled_cpnt_geojson_list[2].on_hover(hover_handler)



        
for i in range(len(styled_cpnt_geojson_list)):
    #add geojson
    mb.add_layer(styled_cpnt_geojson_list[i])


    
#print(styled_cpnt_geojson_list[1]._hover_callbacks)


mb.add_control(LayersControl())
    
mb

Map(center=[19.96323, 110.61723], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

## Mapping of the impact
This part will aime at mapping and characterising the most important impacts that are likely to require important mitigation measures.
This map is not currently available.
### Greenway planning
The environmental impacts of the projects are mainly linked to the restauration of the dyke and creation of a greenway.<br>
![alt text](IMG/MAPS/Hainan_ESMF.png "Planning of the greenway")
*Planning of the greenway*<br><br>
As illustrated by the map below, the proposed greenway crosses sensitive habitats, that could be impacted by the restoration of the dyke. It will be important to gauge the barrier effet of the dyke, and to propose a design that limits hindrances to fauna and flora (e.g. mangrove propagules) across the dyke.

The abandoned pond located westward to the proposed greenway may present interesting opportunities to restore marshes adjacent to the mangrove. It is advantageous from a biodiversity point of view to restore tidal marshes, as these habitats are naturally found in the higher tidal succession of mangroves.<br>
![alt text](IMG/36763-cover.jpg "Abandoned ponds, pioneering marsh characteristics")<br>
*Abandoned ponds, pioneering marsh characteristics*<br><br>
![alt text](IMG/DJI_0514.jpg "Example of a marsh and mangrove succession (Mai po, HK, China)")<br>
*Example of a marsh and mangrove succession (Mai po, HK, China)*<br><br>
We therefore recommend to relocate the component dedicated to shorebird habitat restoration to the west side of the dyke. While the mangrove do play an important role for nesting and roosting of waders, it is important to give the project a expanded scope by realizing the restoration of habitat that can have synergetic interaction with the mangrove. In addition the restoration of ponds inside the mangrove, will not present a great interest for biodiversity as the forest matures the ponds essentially become a micro-habitat within the mangrove ecosystem.
![alt text](IMG/MAPS/BirdHabitat_Move.png "Proposition to relocate the component dedicated to shorebird habitats restorations")<br>
*Proposition to relocate the component dedicated to shorebird habitats restorations*<br><br>
In order to better preserve the near-shore biodiversity sensitive areas, and its natural assets we would recommend to distinguish between a more pedestrian / cycling way directly along the shore and a more traffic-intensive greenway further away for the shore, through the villages, leveraging the existing cultural heritage to develop cultural and eco-tourism.
![alt text](IMG/picture_2022_07_01_08_21_18_856.jpg "Example of architectural heritage in the villages")<br>
*Example of architectural heritage in the villages*<br>

### Seaweed bed restoration projects
Regarding other activities of the project, we note that the seaweed bed restoration project is located in the core area of the park and the mangrove. It is important to understand what will be the protocol deployed in these areas to promote pilot restoration of seaweed beds.

As it is a restoration pilot restoration project, it is indeed interesting to locate it within the protected areas of the bay, as it will allow the project to leverage the management of the park to sustain the long-term success of the project, however it is important to remain cautious when operating in such sensitive areas and details will be needed with regard to the protocol anticipated (e.g. taking examples from similar projects) in order to make sure that they won’t disturb the existing habitats
![alt text](IMG/MAPS/Seaweed_Bed_Impact.png "Example of architectural heritage in the villages")<br>
*Mapping of the seaweed bed restoration project and existing mangrove*<br>

### Mangrove restoration in Sanjiang
Finally, the mangrove restoration in Sanjiang will need to ensure that they respect the natural succession and dynamic of the mangrove as closely as possible. Currently the protocol used for mangrove restoration in the area seem to be lacking the necessary ecological design to ensure the restoration of a the natural mangrove succession occurring in Hainan.<br><br>
Given the short time-frame for the realization of this project, it would be important to cccess to the current protocol in place for the restoration already realized in order to propose lines of improvement upon the propose protocols.<br>
![alt text](IMG/DJI_0445.jpg "Example of current mangrove restoration protocols in Sanjiang park")
<br>
*Example of current mangrove restoration protocols in Sanjiang park*

In [68]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p xyzservices,geopandas,fiona,json,ipywidgets,IPython,ipyleaflet,PIL,watermark


# date
print (" ")

%watermark -u -n -t -z 

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.4.0

xyzservices: 2022.6.0
geopandas  : 0.10.2
fiona      : 1.8.21
json       : 2.0.9
ipywidgets : 7.7.1
IPython    : 8.4.0
ipyleaflet : 0.17.0
PIL        : 9.1.1
watermark  : 2.3.1

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores   : 8
Architecture: 64bit

 
Last updated: Mon Jul 18 2022 01:06:44China Standard Time

